In [1]:
import BioSimSpace as bss

INFO:rdkit:Enabling RDKit 2022.09.1 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [7]:
lig = bss.IO.readMolecules("/home/jguven/projects/metalloenzymes/nonbonded_model_vim2/ligand_6/L8J.mol2")[0]
lig.charge()



-1.0050 |e|

In [3]:
path = "/home/jguven/projects/metalloenzymes/nonbonded_model_vim2/ligand_1/"
lig_mol = bss.IO.readMolecules(path+"L8J.pdb")[0]
lig =  bss.Parameters.gaff2(lig_mol, net_charge=-1).getMolecule()




In [22]:
box_min, box_max = lig.getAxisAlignedBoundingBox()
box_size = [y - x for x, y in zip(box_min, box_max)]
box_sizes = [x + int(20) * bss.Units.Length.angstrom for x in box_size] 
system_box_min, system_box_max = system.getAxisAlignedBoundingBox()
system_box_size = [y - x for x, y in zip(system_box_min, system_box_max)]
system_box_sizes = [x + int(20) * bss.Units.Length.angstrom for x in system_box_size]

In [23]:
print("solvating unbound ligand")
box, angles = bss.Box.cubic(max(box_sizes))
ligand_solvated = bss.Solvent.solvate("tip3p", molecule=lig, box=box, angles=angles)
box, angles = bss.Box.cubic(max(system_box_sizes))
print("solvating bound system")
system_solvated = bss.Solvent.solvate("tip3p", molecule=system, box=box, angles=angles)


solvating unbound ligand
solvating bound system


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   3 ligand_solvated = bss.Solvent.solvate("tip3p", molecule=lig, box=box, angles=angles)         │
│   4 box, angles = bss.Box.cubic(max(system_box_sizes))                                           │
│   5 print("solvating bound system")                                                              │
│ ❱ 6 system_solvated = bss.Solvent.solvate("tip3p", molecule=system, box=box, angles=angles)      │
│   7                                                                                              │
│                                                                                                  │
│ /home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/BioSimSpace/Solvent/_sol │
│ vent.py:127 in solvate                                                                           │
│                                                                                                  │
│    124 │   │   if model not in _models_lower:                                                    │
│    125 │   │   │   raise ValueError("Supported water models are: %s" % waterModels())            │
│    126 │                                                                                         │
│ ❱  127 │   return _model_dict[model](                                                            │
│    128 │   │   molecule, box, angles, shell, ion_conc, is_neutral, work_dir, property_map        │
│    129 │   )                                                                                     │
│    130                                                                                           │
│                                                                                                  │
│ /home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/BioSimSpace/Solvent/_sol │
│ vent.py:382 in tip3p                                                                             │
│                                                                                                  │
│    379 │   )                                                                                     │
│    380 │                                                                                         │
│    381 │   # Create the solvated system.                                                         │
│ ❱  382 │   return _solvate(                                                                      │
│    383 │   │   molecule,                                                                         │
│    384 │   │   box,                                                                              │
│    385 │   │   angles,                                                                           │
│                                                                                                  │
│ /home/jguven/Software/miniconda3/envs/bss-d/lib/python3.9/site-packages/BioSimSpace/Solvent/_sol │
│ vent.py:1058 in _solvate                                                                         │
│                                                                                                  │
│   1055 │   │   │   │   if _isVerbose():                                                          │
│   1056 │   │   │   │   │   raise IOError(msg) from e                                             │
│   1057 │   │   │   │   else:                                                                     │
│ ❱ 1058 │   │   │   │   │   raise IOError(msg) from None                                          │
│   1059 │   │   │                                                                                 │
│   1060 │   │   │   # First write an mdp file.              

In [16]:
bss.IO.saveMolecules("test_lig", ligand_solvated, ["prm7", "rst7"])
bss.IO.saveMolecules("test_sys", system_solvated, ["prm7", "rst7"])  

['/home/jguven/projects/metalloenzymes/scripts/test_sys.prm7',
 '/home/jguven/projects/metalloenzymes/scripts/test_sys.rst7']

In [17]:
bss.IO.saveMolecules("test_sys", system_solvated, ["PDB", "GroTop"])  

['/home/jguven/projects/metalloenzymes/scripts/test_sys.pdb',
 '/home/jguven/projects/metalloenzymes/scripts/test_sys.top']

In [35]:
with open("../kpc2/afe/ligands.dat") as file:
    ligand_lines = file.readlines()
ligand_names = [line.rstrip() for line in ligand_lines]
with open("../kpc2/afe/protocol.dat") as file:
    protocol_lines = file.readlines()

solvent_force_field = protocol_lines[2].split()[-1]
protocol_lines[3].split()[-1].split("*")[0]

'20'

In [3]:
solvent = "TIP3P"
box_edges = 20
box_type = "orthorhombic"
angstrom = bss.Units.Length.angstrom

In [14]:
protein_path = "../kpc2/inputs/protein/"
ligand_path = "../kpc2/inputs/ligands/"
ligand_name_1 = "docked_1.pdb"
ligand_name_2 = "docked_10.pdb"
ligand_1 = bss.IO.readMolecules(ligand_path+ligand_name_1)[0]
ligand_2 = bss.IO.readMolecules(ligand_path+ligand_name_2)[0]
ligand_1_params = bss.Parameters.gaff2(ligand_1, net_charge=-1).getMolecule()
ligand_2_params = bss.Parameters.gaff2(ligand_2, net_charge=-1).getMolecule()

In [ ]:
test = bss.IO.readMolecules([f"../kpc2/inputs/protein/_system_1_solvated.gro",
                                        f"../kpc2/inputs/protein/_system_1_solvated.top"])

In [11]:
bss.IO.fileFormats()

['Gro87', 'GroTop', 'MOL2', 'PDB', 'PRM7', 'PSF', 'RST', 'RST7', 'SDF']

In [5]:
# ligand_params = [ligand_1_params, ligand_2_params]
# ligand_names = [ligand_name_1, ligand_name_2]
# for i in range(2):
#     box_min, box_max = ligand_params[i].getAxisAlignedBoundingBox()
#     box_size = [y - x for x, y in zip(box_min, box_max)]
#     box_sizes = [x + box_edges * angstrom for x in box_size]
#     box, angles = bss.Box.cubic(max(box_sizes))
#     print(f"Box size set to {max(box_sizes)}")
#     print("Solvating ligand.")
#     ligand_params_solvated = bss.Solvent.solvate(solvent, 
#                                                  molecule=ligand_params[i],
#                                                  box=box,
#                                                  angles=angles)
#     print("Solvation done. Saving ligand.")
#     save_name = ligand_names[i].strip(".sdf").split("_")[1]
#     bss.IO.saveMolecules(f"../inputs/ligands/ligand_{save_name}_solv", ligand_params_solvated, ["PRM7", "RST7"])                          
#     bss.IO.saveMolecules(f"../inputs/ligands/ligand_{save_name}_solv", ligand_params_solvated, ["Gro87", "GroTop"])                                            

In [16]:
protein = bss.IO.readMolecules(["../kpc2/inputs/protein/kpc2_tleap.rst7",
                                "../kpc2/inputs/protein/kpc2_tleap.prm7"])[0]
protein.charge()                            
# for i in range(2):
#     system = ligand_params[i] + protein
#     box_min_sys, box_max_sys = system.getAxisAlignedBoundingBox()
#     box_size_sys = [y - x for x, y in zip(box_min_sys, box_max_sys)]
#     box_sizes_sys = [x + box_edges * angstrom for x in box_size_sys]
#     box, angles = bss.Box.cubic(max(box_sizes_sys))
#     print(f"Box size set to {max(box_sizes_sys)}")
#     print("Solvating system")
    
#     system_solvated = bss.Solvent.solvate(solvent,
#                                           molecule=system,
#                                           box=box,
#                                           angles=angles)
#     print(system_solvated.charge())
#     save_name = ligand_names[i].strip(".sdf").split("_")[1]
#     bss.IO.saveMolecules(f"../kpc2/inputs/protein/system_{save_name}_solv", system_solvated, ["PRM7", "RST7"])
#     bss.IO.saveMolecules(f"../kpc2/inputs/protein/system_{save_name}_solv", system_solvated, ["Gro87", "GroTop"])

-1.4433e-07 |e|

In [42]:
protein_charge = protein.charge()
protein_charge

-10.2886 |e|

In [44]:
pt1_charge = bss.Types._charge.Charge(-0.7112999999999998, "e")
pt1_charge

-0.7113 |e|

In [45]:
pt1_charge + protein_charge

-10.9999 |e|